# Scalar [Quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)) of Digital Audio

See https://github.com/vicente-gonzalez-ruiz/quantization.

In [ ]:
%matplotlib inline

import math
import numpy as np
import time

# https://github.com/vicente-gonzalez-ruiz/quantization
!ln -sf ~/quantization/common.py .
!ln -sf ~/quantization/midtread_quantizer.py .
!ln -sf ~/quantization/midrise_quantizer.py .
!ln -sf ~/quantization/companded_quantizer.py .
!ln -sf ~/quantization/deadzone_quantizer.py .

import common
import midtread_quantizer as midtread
import midrise_quantizer as midrise
import deadzone_quantizer as deadzone
import companded_quantizer as companded

## Subjective comparison

In [ ]:
import sounddevice as sd
sr = 44100
duration = 2.0  # seconds
x = sd.rec(int(duration * sr), samplerate=sr, channels=1, dtype=np.int16)
print("Speak!")
while sd.wait():
    pass
print("done")

In [ ]:
sd.play(x)
common.plot(np.linspace(0, len(x)-1, len(x)), x, "Time", "Amplitude", "Audio Signal")

In [ ]:
len(x)

In [ ]:
quantization_step = 10000

In [ ]:
y_T, k_T  = midtread.quan_dequan(x, quantization_step)
y_T = y_T.astype(np.int16)  # soundevice process up to 16 bits/sample
y_R, k_R  = midrise.quan_dequan(x, quantization_step)
y_R = y_R.astype(np.int16)
y_D, k_D = deadzone.quan_dequan(x, quantization_step)
y_D = y_D.astype(np.int16)
y_C, k_C  = companded.quan_dequan(x, quantization_step)
y_C = y_C.astype(np.int16)

In [ ]:
sd.play(y_T)
common.plot(np.linspace(0, len(y_T)-1, len(y_T)), y_T, "Time", "Amplitude", "Mid-tread ($\Delta={}$)".format(quantization_step))
time.sleep(duration)

In [ ]:
sd.play(y_R)
common.plot(np.linspace(0, len(y_R)-1, len(y_R)), y_R, "Time", "Amplitude", "Mid-rise ($\Delta={}$)".format(quantization_step))
time.sleep(duration)

In [ ]:
sd.play(y_D)
common.plot(np.linspace(0, len(y_D)-1, len(y_D)), y_D, "Time", "Amplitude", "Dead-zone ($\Delta={}$)".format(quantization_step))
time.sleep(duration)

In [ ]:
sd.play(y_C)
common.plot(np.linspace(0, len(y_C)-1, len(y_C)), y_C, "Time", "Amplitude", "Companded Dead-zone ($\Delta={}$)".format(quantization_step))

In [ ]:
common.plot(np.linspace(0, len(k_T)-1, len(k_T)), k_T, "Time", "Representation Codes", "Mid-tread ($\Delta={}$)".format(quantization_step))
common.plot(np.linspace(0, len(k_R)-1, len(k_R)), k_R, "Time", "Representation Codes", "Mid-rise ($\Delta={}$)".format(quantization_step))
common.plot(np.linspace(0, len(k_D)-1, len(k_D)), k_D, "Time", "Representation Codes", "Dead-zone ($\Delta={}$)".format(quantization_step))
common.plot(np.linspace(0, len(k_C)-1, len(k_C)), k_C, "Time", "Representation Codes", "Companded Dead-zone ($\Delta={}$)".format(quantization_step))

In [ ]:
error_T = x - y_T
error_R = x - y_R
error_D = x - y_D
error_C = x - y_C

In [ ]:
sd.play(error_T)
common.plot(np.linspace(0, len(y_T)-1, len(y_T)), error_T, "Time", "Amplitude Error", "Mid-tread ($\Delta={}$)".format(quantization_step))
time.sleep(duration)

In [ ]:
sd.play(error_R)
common.plot(np.linspace(0, len(y_R)-1, len(y_R)), error_R, "Time", "Amplitude Error", "Mid-rise ($\Delta={}$)".format(quantization_step))
time.sleep(duration)

In [ ]:
sd.play(error_D)
common.plot(np.linspace(0, len(y_D)-1, len(y_D)), error_D, "Time", "Amplitude Error", "Dead-zone ($\Delta={}$)".format(quantization_step))
time.sleep(duration)

In [ ]:
sd.play(error_C)
common.plot(np.linspace(0, len(y_C)-1, len(y_C)), error_C, "Time", "Amplitude Error", "Companded Dead-zone ($\Delta={}$)".format(quantization_step))

## Objective comparison (Rate/Distortion curve)

In [ ]:
!ln -sf ~/quantization/information.py .
!cat information.py
import information

In [ ]:
!ln -sf ~/quantization/distortion.py .
!cat distortion.py
import distortion

### RMSE vs bit-rate

In [ ]:
def RD_curve(x, qdeq):
    points = []
    for q_step in range(1, 32768, 256):
        y, k = qdeq(x, q_step)
        rate = information.entropy(k)
        distortion_ = distortion.RMSE(x, y)
        points.append((rate, distortion_))
        print(rate, distortion_)
    return points

In [ ]:
midtread_RD_points = RD_curve(x, midtread.quan_dequan)

In [ ]:
midrise_RD_points = RD_curve(x, midrise.quan_dequan)

In [ ]:
deadzone_RD_points = RD_curve(x, deadzone.quan_dequan)

In [ ]:
companded_RD_points = RD_curve(x, companded.quan_dequan)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
plt.title("RD Tradeoff")
plt.xlabel("Bits per Sample")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*midtread_RD_points), s=2, c='b', marker="o", label='Mid-tread')
plt.scatter(*zip(*midrise_RD_points), s=2, c='c', marker="o", label='Mid-rise')
plt.scatter(*zip(*deadzone_RD_points), s=2, c='r', marker="o", label='Dead-zone')
plt.scatter(*zip(*companded_RD_points), s=2, c='g', marker="o", label='Companded Dead-zone')
plt.legend(loc='upper right')
plt.gcf().set_size_inches(1.0 * plt.gcf().get_size_inches())
plt.show()

In [ ]:
def QstepD_curve(x, qdeq):
    QstepD_points = []
    for q_step in range(1, 32768, 256):
        y, k = qdeq(x, q_step)
        distortion_ = distortion.RMSE(x, y)
        QstepD_points.append((q_step, distortion_))
        print(q_step, distortion_)
    return QstepD_points

In [ ]:
midtread_QstepD_points = QstepD_curve(x, midtread.quan_dequan)

In [ ]:
midrise_QstepD_points = QstepD_curve(x, midrise.quan_dequan)

In [ ]:
deadzone_QstepD_points = QstepD_curve(x, deadzone.quan_dequan)

In [ ]:
companded_QstepD_points = QstepD_curve(x, companded.quan_dequan)

In [ ]:
plt.title("$\Delta$/D comparative")
plt.xlabel("$\Delta$")
plt.ylabel("RMSE")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*midtread_QstepD_points), s=2, c='b', marker="o", label='Mid-tread')
plt.scatter(*zip(*midrise_QstepD_points), s=2, c='c', marker="o", label='Mid-rise')
plt.scatter(*zip(*deadzone_QstepD_points), s=2, c='r', marker="o", label='Dead-zone')
plt.scatter(*zip(*companded_QstepD_points), s=2, c='g', marker="o", label='Companded Dead-zone')
plt.legend(loc='upper right')
plt.show()

In general, we can say that the distortion grows logarithmically with the quantization step.

In [ ]:
def QstepR_curve(x, qdeq):
    points = []
    for q_step in range(1, 32768, 256):
        y, k = qdeq(x, q_step)
        rate = information.entropy(k)
        points.append((q_step, rate))
        print(q_step, rate)
    return points

In [ ]:
midtread_QstepR_points = QstepR_curve(x, midtread.quan_dequan)

In [ ]:
midrise_QstepR_points = QstepR_curve(x, midrise.quan_dequan)

In [ ]:
deadzone_QstepR_points = QstepR_curve(x, deadzone.quan_dequan)

In [ ]:
companded_QstepR_points = QstepR_curve(x, companded.quan_dequan)

In [ ]:
plt.title("$\Delta$/R comparative")
plt.xlabel("$\Delta$")
plt.ylabel("Bits per sample")
#plt.xscale("log")
#plt.yscale("log")
plt.scatter(*zip(*midtread_QstepR_points), s=2, c='b', marker="o", label='Mid-tread')
plt.scatter(*zip(*midrise_QstepR_points), s=2, c='c', marker="o", label='Mid-rise')
plt.scatter(*zip(*deadzone_QstepR_points), s=2, c='r', marker="o", label='Dead-zone')
plt.scatter(*zip(*companded_QstepR_points), s=2, c='g', marker="o", label='Companded Dead-zone')
plt.legend(loc='upper right')
plt.show()

In general, we can say that the bit-rate decreases logarithmically with the quantization step.

### Using a logaritmic version of RMSE

The [HAS](https://en.wikipedia.org/wiki/Auditory_system) is more sensitive to the variations of quiet sounds than to the variations of the loud sounds. For this reason, let's see what happens when we give more importance to the quiet sounds that to the loud sounds, using a compression of the dynamic range of the sounds.

In [ ]:
def log_average_energy(x):
    '''In fact, average logaritmic energy.'''
    return np.sum(np.log(np.abs(x.astype(np.double))+1)*np.log(np.abs(x.astype(np.double))+1))/len(x)

In [ ]:
def log_RMSE(x, y):
    error_signal = x - y
    return math.sqrt(log_average_energy(error_signal))

In [ ]:
def log_RD_curve(x, qdeq):
    RD_points = []
    for q_step in range(1, 32768, 256):
        y, k = qdeq(x, q_step)
        rate = information.entropy(k)
        distortion_ = log_RMSE(x, y)
        RD_points.append((rate, distortion_))
        print(rate, distortion_)
    return RD_points

In [ ]:
log_midtread_RD_points = log_RD_curve(x, midtread.quan_dequan)

In [ ]:
log_midrise_RD_points = log_RD_curve(x, midrise.quan_dequan)

In [ ]:
log_deadzone_RD_points = log_RD_curve(x, deadzone.quan_dequan)

In [ ]:
log_companded_RD_points = log_RD_curve(x, companded.quan_dequan)

In [ ]:
plt.title("R/D comparative")
plt.xlabel("Bits per sample")
plt.ylabel("(log) RMSE")
#plt.yscale("linear")
#plt.xscale("log")
plt.xlim(1, 16)
plt.plot(*zip(*log_midtread_RD_points), c='b', marker="o", label='Mid-tread')
plt.plot(*zip(*log_midrise_RD_points), c='c', marker="o", label='Mid-rise')
plt.plot(*zip(*log_deadzone_RD_points), c='r', marker="o", label='Dead-zone')
plt.plot(*zip(*log_companded_RD_points), c='g', marker="o", label='Companded Dead-zone')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Could be useful if we replace RMSE by SNR
def SNR(x, y):
    signal_energy = compute_average_energy(x)
    error_energy = compute_average_energy(x-y)
    print("signal energy =", signal_energy)
    print("error energy =", error_energy)
    return 10*math.log(signal_energy/error_energy)